In [1]:
import ray
ray.init(address="localhost:6379")

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.10', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '192.168.2.233', 'raylet_ip_address': '192.168.2.233', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-12_13-56-03_219307_916984/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-12_13-56-03_219307_916984/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-05-12_13-56-03_219307_916984', 'metrics_export_port': 64671, 'gcs_address': '192.168.2.233:6379', 'address': '192.168.2.233:6379', 'node_id': '98cd0ff4b6126c06e2e7afb8030b5c048f70317a15218b04e73697ac'})

In [2]:
import numpy as np
import pandas as pd
import sklearn

import tune_sklearn

In [3]:
from sklearn.datasets import fetch_covtype

forest = fetch_covtype()

In [4]:
from sklearn.model_selection import train_test_split

N = 10000
features = forest['data'][:N]
labels = forest['target'][:N]

X_train, X_test, y_train, y_test = train_test_split(
     features, labels, train_size=0.75, test_size=0.25
)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from tune_sklearn import TuneGridSearchCV

clf = RandomForestClassifier()
param_grid = { 
    'n_estimators': [200, 500],
    'max_depth' : [4,5,6,7,8],
    'ccp_alpha': [0,0.2,0.4,0.6,0.8,1]
}

tune_search = TuneGridSearchCV(
    estimator=clf,
    param_grid=param_grid
)

In [6]:
import time 

start = time.time()
tune_search.fit(X_train, y_train)
end = time.time()
print("Tune Fit Time:", end - start)

pred = tune_search.predict(X_test)
accuracy = np.count_nonzero(np.array(pred) == np.array(y_test)) / len(pred)
print("Tune Accuracy:", accuracy)

2022-05-12 13:56:42,562	INFO trial_runner.py:803 -- starting _Trainable_596cb_00000
2022-05-12 13:56:43,374	INFO trial_runner.py:803 -- starting _Trainable_596cb_00001
2022-05-12 13:56:50,040	INFO trial_runner.py:803 -- starting _Trainable_596cb_00002
2022-05-12 13:56:51,025	INFO trial_runner.py:803 -- starting _Trainable_596cb_00003
2022-05-12 13:56:54,988	INFO trial_runner.py:803 -- starting _Trainable_596cb_00004
2022-05-12 13:56:55,823	INFO trial_runner.py:803 -- starting _Trainable_596cb_00005
2022-05-12 13:56:59,866	INFO trial_runner.py:803 -- starting _Trainable_596cb_00006
2022-05-12 13:57:00,648	INFO trial_runner.py:803 -- starting _Trainable_596cb_00007
2022-05-12 13:57:05,221	INFO trial_runner.py:803 -- starting _Trainable_596cb_00008
2022-05-12 13:57:05,944	INFO trial_runner.py:803 -- starting _Trainable_596cb_00009
2022-05-12 13:57:10,628	INFO trial_runner.py:803 -- starting _Trainable_596cb_00010
2022-05-12 13:57:11,169	INFO trial_runner.py:803 -- starting _Trainable_596c

Tune Fit Time: 314.3112018108368
Tune Accuracy: 0.7604


In [7]:
tune_search.best_params_

{'n_estimators': 200, 'max_depth': 8, 'ccp_alpha': 0}